In [2]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

  Using cached tensorflow-2.4.1-cp38-cp38-win_amd64.whl (370.7 MB)
  Using cached sklearn-0.0-py2.py3-none-any.whl
  Using cached numpy-1.19.5-cp38-cp38-win_amd64.whl (13.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.4


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\janbinkowski\\desktop\\signlanguagerecognition\\venv\\lib\\site-packages\\numpy-1.21.4.dist-info\\RECORD'

You should consider upgrading via the 'C:\Users\JanBinkowski\Desktop\SignLanguageRecognition\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [4]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

ImportError: cannot import name 'LayerNormalization' from 'tensorflow.python.keras.layers.normalization' (c:\users\janbinkowski\desktop\signlanguagerecognition\venv\lib\site-packages\tensorflow\python\keras\layers\normalization\__init__.py)

In [ ]:
# Path to data for train
DATA_PATH = os.path.join('C://Users//JanBinkowski//Desktop//SignLanguageRecognition//MP_DATA')

# Actions to detect
# actions = np.array(['a', 'b', 'c'])
actions = np.array(open('C://Users//JanBinkowski//Desktop//SignLanguageRecognition//classes.txt', 'r').read().split('\n'))
# Thirty videos worth of data
no_sequences = 100

# frames for one sequence
sequence_length = 30

In [ ]:
sequences, labels = [], []

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
X = np.array(sequences)

In [ ]:
X_sliced_for_training = np.concatenate((X[:80],X[100:180],X[200:280]), axis=0)

In [ ]:
X_sliced_for_validation = np.concatenate((X[80:100],X[180:200],X[280:300]), axis=0)

In [ ]:
# X_sliced_for_testing = np.concatenate((X[90:100],X[190:200],X[290:300]), axis=0)

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
y_sliced_for_training = np.concatenate((y[:80],y[100:180],y[200:280]), axis=0)

In [ ]:
y_sliced_for_validation = np.concatenate((y[80:100],y[180:200],y[280:300]), axis=0)

In [ ]:
# y_sliced_for_testing = np.concatenate((y[90:100],y[190:200],y[290:300]), axis=0)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X_sliced_for_training, y_sliced_for_training, test_size=0.05)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('C://Users//JanBinkowski//Desktop//SignLanguageRecognition//Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
# model.fit(X_train, y_train, epochs=200, batch_size=4, validation_data=(X_sliced_for_validation, y_sliced_for_validation), callbacks=[tb_callback])
model.fit(X_train, y_train, epochs=200, validation_split=0.2, callbacks=[tb_callback])


In [ ]:
model.summary()

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=1)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 30 samples")
predictions = model.predict(X_test)
print("predictions shape:", predictions.shape)

In [ ]:
model.save('C://Users//JanBinkowski//Desktop//SignLanguageRecognition//Weights//action.h5')